# Imports, install and mount

In [42]:
import play_with_complex as pwc
import random
import torch
import torch.optim as optim
import torch_geometric
from torch_geometric.nn import ComplEx
from torch_geometric.data import Data
import pandas as pd
from torch_geometric.loader import DataLoader
from nxontology.imports import from_file
import pickle
import wandb
import torch.nn.functional as F
import numpy as np


# Settings 


In [43]:
# Datas
mapped_iric_path = '/home/ebutz/ESL2024/data/little_mapped_iric.tsv'
datasets_save_path = '/home/elliot/Documents/ESL2024/data/mapping_datasets_and_model_for_genes_to_phenotypes_iric/test_dataset_'

val_path = datasets_save_path + 'VAL' +  '.pickle'
test_path = datasets_save_path + 'TEST' +  '.pickle'
train_path = datasets_save_path + 'TRAIN' +  '.pickle'

# Model to load :
hidden_channels = 220

params_save_name = f"PARAMS_ComplEx_6_times_{hidden_channels}_HC_{500}_epochs_{4096}_BS_on_full_Os_GO"
model_parameters_path = "/home/elliot/Documents/ESL2024/data/mapping_datasets_and_model_for_genes_to_phenotypes_iric/"+params_save_name


# Model to re-train :
hidden_channels = 220
batch_size = 4096
epochs = 500
lin_factor = 0.5

params_save_name = f"PARAMS_ComplEx_6_times_{hidden_channels}_HC_{epochs}_epochs_{batch_size}_BS_on_full_Os_GO"
model_parameters_path = "/home/elliot/Documents/ESL2024/data/mapping_datasets_and_model_for_genes_to_phenotypes_iric/"+params_save_name

# Ontology
url = "/home/elliot/Documents/ESL2024/data/go-basic.json.gz"

# Device
device = 'cuda' if torch.cuda.is_available() else 'cpu'
# device = 'cpu' # Tip : Use cpu for debugging
print(device)

cuda


In [44]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(device)
a = torch.Tensor([0,1])
print(a.is_cuda)
a = a.to(device)
print(a.is_cuda)

cuda
False
True


# data loading


In [45]:
mapped_iric = pd.read_csv(mapped_iric_path, sep = '\t', names=['subject', 'predicate', 'object', 'mapped_subject', 'mapped_predicate', 'mapped_object','mapped_alt_tails'])
display(mapped_iric)

GO_to_map = mapped_iric.set_index('object')['mapped_object'].to_dict()
map_to_GO = {value: key for key, value in GO_to_map.items()}


# Checking dict :
looks_ok: bool = True
print(len(list(mapped_iric['object'])))
for i in range(len(list(mapped_iric['object']))):
    if GO_to_map[mapped_iric['object'][i]]!=mapped_iric['mapped_object'][i]:
        looks_ok = False
    
print('Dict looks ok :', looks_ok)
print(map_to_GO)
print(GO_to_map)

,subject,predicate,object,mapped_subject,mapped_predicate,mapped_object,mapped_alt_tails
0,subject,predicate,object,mapped_subject,mapped_predicate,mapped_object,NaN
1,OsNippo01g010050,gene ontology,GO:0031267,26438,0,5690,NaN
2,OsNippo01g010050,gene ontology,GO:0006886,26438,0,9447,NaN
3,OsNippo01g010050,gene ontology,GO:0005622,26438,0,20316,NaN
4,OsNippo01g010050,gene ontology,GO:0005623,26438,0,10727,NaN
...,...,...,...,...,...,...,...
9996,OsNippo01g223000,gene ontology,GO:0005085,14223,0,13297,NaN
9997,OsNippo01g223000,gene ontology,GO:0005784,14223,0,22814,NaN
9998,OsNippo01g223050,gene ontology,GO:0005634,28984,0,3372,NaN
9999,OsNippo01g223050,gene ontology,GO:0005737,28984,0,23833,NaN


10001
Dict looks ok : True
{'mapped_object': 'object', '5690': 'GO:0031267', '9447': 'GO:0006886', '20316': 'GO:0005622', '10727': 'GO:0005623', '6491': 'GO:0090630', '27241': 'GO:0043087', '21142': 'GO:0005096', '8515': 'GO:0020037', '22717': 'GO:0016705', '24275': 'GO:0055114', '8816': 'GO:0004497', '8471': 'GO:0005506', '10504': 'GO:0009055', '5257': 'GO:0016722', '24798': 'GO:0005507', '13721': 'GO:0016491', '26683': 'GO:0005886', '19374': 'GO:0009506', '30373': 'GO:0046658', '12164': 'GO:0016020', '13511': 'GO:0003676', '20542': 'GO:0006412', '3376': 'GO:0019843', '14915': 'GO:0003723', '9030': 'GO:0042542', '24245': 'GO:0000028', '12271': 'GO:0022627', '9670': 'GO:0009651', '11545': 'GO:0003735', '17521': 'GO:0009414', '28472': 'GO:0009737', '27704': 'GO:0050832', '5167': 'GO:0003729', '28663': 'GO:0005840', '567': 'GO:0015935', '5898': 'GO:0005783', '20917': 'GO:0016829', '558': 'GO:0006888', '19953': 'GO:0051788', '3767': 'GO:0006629', '6430': 'GO:0006635', '6644': 'GO:0043161'

In [46]:
from tqdm import tqdm
tqdm.pandas()

mapped_iric['mapped_alt_tails'] = mapped_iric.progress_apply(
                                             lambda row: 
                                             mapped_iric.loc[(mapped_iric['mapped_subject'] == row['mapped_subject']) & 
                                                             (mapped_iric['mapped_predicate'] == row['mapped_predicate']) & 
                                                             (mapped_iric['mapped_object'] != row['mapped_object']), 
                                                             'mapped_object'].values, 
                                             axis=1)
display(mapped_iric)

  0%|          | 0/10001 [00:00<?, ?it/s]

100%|██████████| 10001/10001 [00:22<00:00, 439.75it/s]


,subject,predicate,object,mapped_subject,mapped_predicate,mapped_object,mapped_alt_tails
0,subject,predicate,object,mapped_subject,mapped_predicate,mapped_object,[]
1,OsNippo01g010050,gene ontology,GO:0031267,26438,0,5690,"[9447, 20316, 10727, 6491, 27241, 21142]"
2,OsNippo01g010050,gene ontology,GO:0006886,26438,0,9447,"[5690, 20316, 10727, 6491, 27241, 21142]"
3,OsNippo01g010050,gene ontology,GO:0005622,26438,0,20316,"[5690, 9447, 10727, 6491, 27241, 21142]"
4,OsNippo01g010050,gene ontology,GO:0005623,26438,0,10727,"[5690, 9447, 20316, 6491, 27241, 21142]"
...,...,...,...,...,...,...,...
9996,OsNippo01g223000,gene ontology,GO:0005085,14223,0,13297,"[25433, 5560, 5898, 9447, 12164, 22985, 20156,..."
9997,OsNippo01g223000,gene ontology,GO:0005784,14223,0,22814,"[25433, 5560, 5898, 9447, 12164, 22985, 20156,..."
9998,OsNippo01g223050,gene ontology,GO:0005634,28984,0,3372,"[23833, 13511]"
9999,OsNippo01g223050,gene ontology,GO:0005737,28984,0,23833,"[3372, 13511]"


In [47]:
display(mapped_iric)

,subject,predicate,object,mapped_subject,mapped_predicate,mapped_object,mapped_alt_tails
0,subject,predicate,object,mapped_subject,mapped_predicate,mapped_object,[]
1,OsNippo01g010050,gene ontology,GO:0031267,26438,0,5690,"[9447, 20316, 10727, 6491, 27241, 21142]"
2,OsNippo01g010050,gene ontology,GO:0006886,26438,0,9447,"[5690, 20316, 10727, 6491, 27241, 21142]"
3,OsNippo01g010050,gene ontology,GO:0005622,26438,0,20316,"[5690, 9447, 10727, 6491, 27241, 21142]"
4,OsNippo01g010050,gene ontology,GO:0005623,26438,0,10727,"[5690, 9447, 20316, 6491, 27241, 21142]"
...,...,...,...,...,...,...,...
9996,OsNippo01g223000,gene ontology,GO:0005085,14223,0,13297,"[25433, 5560, 5898, 9447, 12164, 22985, 20156,..."
9997,OsNippo01g223000,gene ontology,GO:0005784,14223,0,22814,"[25433, 5560, 5898, 9447, 12164, 22985, 20156,..."
9998,OsNippo01g223050,gene ontology,GO:0005634,28984,0,3372,"[23833, 13511]"
9999,OsNippo01g223050,gene ontology,GO:0005737,28984,0,23833,"[3372, 13511]"


In [50]:
mapped_alt_tails = {}

for index, row in mapped_iric.iterrows():
    key = (row['mapped_subject'], row['mapped_predicate'])
    if key not in mapped_alt_tails:
        mapped_alt_tails[key] = set()
    mapped_alt_tails[key].update(set(row['mapped_alt_tails']))

print(mapped_alt_tails)

for key, value in mapped_alt_tails.items():
    mapped_alt_tails[key]=np.array(list(value))

print(mapped_alt_tails[(30291, 0)])

{('mapped_subject', 'mapped_predicate'): set(), ('26438', '0'): {'9447', '20316', '27241', '21142', '5690', '6491', '10727'}, ('139', '0'): {'22717', '8515', '8471', '24275', '8816', '10504'}, ('15161', '0'): {'5257', '24275', '19374', '30373', '13721', '24798', '26683'}, ('24076', '0'): set(), ('27081', '0'): set(), ('1774', '0'): set(), ('13031', '0'): {'3376', '9030', '20542', '567', '11545', '20316', '28472', '5167', '14915', '28663', '27704', '9670', '24245', '17521', '12271'}, ('3880', '0'): {'20847', '19431', '5898', '10648', '5077', '983', '558', '19953', '23879', '9423', '4565', '21072', '13673', '3767', '20029', '12164', '24016', '20917', '12756', '6644', '6430', '27704'}, ('5741', '0'): {'23833', '27968', '3372', '14915'}, ('16615', '0'): {'4563', '21964', '14915', '24083', '13511', '11737'}, ('28876', '0'): {'22704', '6483', '19847', '5898', '19309', '3808', '3372', '1304', '23486'}, ('13153', '0'): set(), ('20842', '0'): {'8119', '25982', '3808', '18372', '23226', '11982',

KeyError: (30291, 0)

In [51]:
val_data = torch.load(val_path)
test_data = torch.load(test_path)
train_data = torch.load(train_path)

print("Datatsets look OK ? (val, train, test) :",
val_data.validate(),
test_data.validate(),
train_data.validate())

print(val_data)
print(test_data)
print(train_data)

FileNotFoundError: [Errno 2] No such file or directory: '/home/elliot/Documents/ESL2024/data/mapping_datasets_and_model_for_genes_to_phenotypes_iric/test_dataset_VAL.pickle'

In [ ]:
subs = set(list(mapped_iric['mapped_subject']))
objs = set(list(mapped_iric['mapped_object']))

heads = set(train_data.edge_index[0].tolist())
tails = set(train_data.edge_index[1].tolist())

print('Number of heads in dataset  :', len(heads))
print('difference heads - subjects :', len(heads-subs))
print('difference heads - object   :', len(heads-objs))

print('Numer of tails in dataset   :', len(tails))
print('Difference tails - subjects :', len(tails-subs))
print('Differece tails - objects   :', len(tails-objs))

Number of heads in dataset  : 22648
difference heads - subjects : 0
difference heads - object   : 22648
Numer of tails in dataset   : 6557
Difference tails - subjects : 6557
Differece tails - objects   : 0


In [ ]:
# Here I create a batch on wich i will test my losses. I need a loader and a model to create it.

complex_model = ComplEx(
    num_nodes=train_data.num_nodes,
    num_relations = train_data.edge_index.size()[1],
    hidden_channels=hidden_channels,
).to(device)

loader = complex_model.loader(
    head_index = train_data.edge_index[0],
    tail_index = train_data.edge_index[1],
    rel_type = train_data.edge_attr,
    batch_size=batch_size,
    shuffle=False,)

batchy = next(iter(loader))
hi, rt, ti = batchy[0], batchy[1], batchy[2]
print('\n',hi,'\n',rt,'\n', ti)


 tensor([12043,  2912, 16271,  ...,   478,  9393, 27290]) 
 tensor([0, 0, 0,  ..., 0, 0, 0]) 
 tensor([16231,  1781, 13382,  ..., 18714,  3859,   157])


# Defining LinLoss() functions

In [ ]:
nxo = from_file('/home/elliot/Documents/ESL2024/data/go-basic.json.gz')
nxo.freeze()
print(nxo)

# Defining training functions

# Initiating models

In [49]:

print(f"Device : {device}, batch size : {batch_size}, epochs : {epochs}, hidden channels : {hidden_channels}")
print("DataSets :", '\n',train_data,'\n', test_data,'\n', val_data)

print("\nModel types :\n  -ComplEx ;\n  -Tail only Complex ;\n  -ComplEx with LinSim ;\n  -ComplEx with best possible LinSim")


usual_complex = ComplEx(
    num_nodes = train_data.num_nodes,
    num_relations = train_data.edge_index.size()[1],
    hidden_channels = hidden_channels,
)

to_complex = pwc.tail_only_ComplEx(
    num_nodes=train_data.num_nodes,
    num_relations = train_data.edge_index.size()[1],
    hidden_channels=hidden_channels,
)

lin_complex = pwc.LinSim_ComplEx(
    num_nodes=train_data.num_nodes,
    num_relations = train_data.edge_index.size()[1],
    hidden_channels=hidden_channels,
)

lin_only_complex = pwc.LinSim_Only_ComplEx(
    num_nodes=train_data.num_nodes,
    num_relations = train_data.edge_index.size()[1],
    hidden_channels=hidden_channels,
)

noise_plus_usual_labels_complex = pwc.ComplEx_with_normal_noise_and_usual_labels(
    num_nodes = train_data.num_nodes,
    num_relations = train_data.edge_index.size()[1],
    hidden_channels = hidden_channels,
)

models = {
    'classic':usual_complex,
    'tail_only':to_complex,
    'additionnal_lin_term':lin_complex,
}

Device : cuda, batch size : 4096, epochs : 500, hidden channels : 220


NameError: name 'train_data' is not defined

In [ ]:
device

'cpu'

In [ ]:
classic, usual_train_losses, usual_test_losses = pwc.train_and_test_complex(model=usual_complex,
                                                                    train_data=train_data, test_data=test_data,
                                                                    xp_name='Pipeline with train losses',
                                                                    epochs=1000,
                                                                    batch_size=2000,
                                                                    eval_period=0,
                                                                    reset_parameters=True,
                                                                    save_params=False,
                                                                    use_wandb = False,
                                                                    device=device
                                                                    
                                                                    )

SyntaxError: invalid syntax (3648410938.py, line 11)